# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [3]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------

# For feature selection
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, f_classif

# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [4]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [5]:
# word net installation:

# unmark if you want to use and need to install
#!pip install wn
#!python -m wn download omw-he:1.4

In [6]:
# word net import:

# unmark if you want to use:
#import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [7]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [8]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\Admin\Downloads\assignment5-text_analysis


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [9]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [10]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [11]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [12]:
# Omer Elazar

In [13]:
def Tokenizer(text): # Use hebrew tokenizer to get list of hebrew words from text
    tokens = ht.tokenize(text)
    toks = [token for grp, token, token_num, (start_index, end_index) in tokens if grp == 'HEBREW' and end_index - start_index > 2]
    return toks

def clean_text(text):
    text = re.sub(r'\d+|[^\w\s]', '', text) # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces
    text = re.sub(r'כש+', '', text) # Remove כש prefix
    return text

def count_taf(text): # Count words ending with ת
    return len(re.findall("[א-ת]+ת ", text, flags=re.I))

def count_punct(text): # Count punctuations
    return len(re.findall('[^\w\s]', text))

In [14]:
# Counting and cleaning
df_train['taf'] = df_train['story'].apply(count_taf)
df_train['punct'] = df_train['story'].apply(count_punct)
df_train['story'] = df_train['story'].apply(clean_text)

# Encode gender column (m, f) to numeric (1, 0)
le = LabelEncoder()
df_train['gender'] = le.fit_transform(df_train['gender'])
df_train

,story,gender,taf,punct
0,חבר הזמין אותי לחול לא באמת חשבתי שזה יקרה פשו...,1,30,90
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,1,25,48
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,0,36,26
3,הייתי ילד מטוסים היה הדבר שהכי ריתק אותי בתור ...,1,29,74
4,הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכים...,0,29,34
...,...,...,...,...
748,אז לפני שנה בדיוק טסתי לאמסטרדם עם שני חברים ט...,1,18,39
749,שבוע שעבר העליתי באופן ספונטני רעיון לנסוע עם ...,1,43,37
750,לפני חודש עברנו לדירה בבית שמש בעקבות משפחתי ה...,1,25,11
751,החוויה אותה ארצה לשתף התרחשה לפני כמה חודשים ז...,0,21,56


In [15]:
# Splitting feature and target columns
X_train = df_train['story']
y_train = df_train['gender']

Preprocessing the data (Vectorization, Feature selection)

In [235]:
# Vectorizing the texts using the tfidf vectorizer
tfidf = TfidfVectorizer(sublinear_tf=False, min_df=5, max_df=0.9, ngram_range=(1, 1), max_features=5000, tokenizer=Tokenizer)
X_train_tfidf = tfidf.fit_transform(X_train).toarray()

# Vectorizing the texts using the count vectorizer
count = CountVectorizer(min_df=5, max_df=0.9, ngram_range=(1, 2), max_features=10000, tokenizer=Tokenizer)
X_train_count = count.fit_transform(X_train).toarray()

# Adding the ת and punctuation count columns
X_train_tfidf = np.column_stack((X_train_tfidf, df_train['taf']))
X_train_tfidf = np.column_stack((X_train_tfidf, df_train['punct']))

X_train_count = np.column_stack((X_train_count, df_train['taf']))
X_train_count = np.column_stack((X_train_count, df_train['punct']))

# Using SelectKBest feature selector to get the k highest scoring features according to a scoring function,
# in this case using the default function (f_classif)
selector = SelectKBest(k=1000)
X_train__tfidf_selected = selector.fit_transform(X_train_tfidf, y_train)
X_train_count_selected = selector.fit_transform(X_train_count, y_train)

In [162]:
models = {'DecisionTreeClassifier': DecisionTreeClassifier(),
          'GaussianNB': GaussianNB(),
          'KNeighborsClassifier': KNeighborsClassifier(),
          'MLPClassifier':MLPClassifier(),
          'Perceptron':Perceptron(),
          'MultinomialNB': MultinomialNB()}

In [163]:
def get_cv_score(models, X, y): # Print average cross validation score for each of the estimators
    print('Avg f1 macro:')
    for name, estimator in models.items():
        cvs = cross_val_score(estimator , X, y, cv=5, scoring='f1_macro')
        print('{}: {}'.format(name, round(cvs.mean(), 2)))

Comparing the performance of the models on the two vectorized datasets (tfidf and count) both before and after feature selection

In [165]:
get_cv_score(models, X_train__tfidf_selected, y_train)

Avg f1 macro:
DecisionTreeClassifier: 0.58
GaussianNB: 0.82
KNeighborsClassifier: 0.43
MLPClassifier: 0.92
Perceptron: 0.76
MultinomialNB: 0.43


In [166]:
get_cv_score(models, X_train_count_selected, y_train)

Avg f1 macro:
DecisionTreeClassifier: 0.65
GaussianNB: 0.85
KNeighborsClassifier: 0.46
MLPClassifier: 0.88
Perceptron: 0.85
MultinomialNB: 0.93


In [111]:
get_cv_score(models, X_train_tfidf, y_train)

Avg f1 macro:
DecisionTreeClassifier: 0.6
GaussianNB: 0.49
KNeighborsClassifier: 0.48
MLPClassifier: 0.6
Perceptron: 0.35
MultinomialNB: 0.44


In [112]:
get_cv_score(models, X_train_count, y_train)

Avg f1 macro:
DecisionTreeClassifier: 0.6
GaussianNB: 0.48
KNeighborsClassifier: 0.5
MLPClassifier: 0.62
Perceptron: 0.67
MultinomialNB: 0.66


The highest scoring model is the MultinomilNB on the count vectorized and feature selected dataset with an average f1 macro score of 0.93.

Next I will use GridSearchCV to find the best parameters for the vectorizer, selector and estimator.

In [126]:
pipe = Pipeline([('vec', CountVectorizer()), 
                 ('slct', SelectKBest()),
                 ('clf', MultinomialNB())])

pipe_params = {'vec__min_df': [1, 5],
              'vec__max_df': [0.8, 0.9, 1.0],
              'vec__ngram_range': [(1, 1), (1, 2)],
              'vec__max_features': [10000], # To improve computation time
              'vec__tokenizer': [Tokenizer, None],
              'slct__score_func': [chi2, mutual_info_classif, f_classif],
              'slct__k': [500, 1000, 2000],
              'clf__alpha': np.logspace(0,-4, num=5),
              'clf__fit_prior': [True, False]}

grid = GridSearchCV(pipe, pipe_params, cv=5, scoring='f1_macro', n_jobs=-1, verbose=2)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 2160 candidates, totalling 10800 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vec', CountVectorizer()),
                                       ('slct', SelectKBest()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04]),
                         'clf__fit_prior': [True, False],
                         'slct__k': [500, 1000, 2000],
                         'slct__score_func': [<function chi2 at 0x000002799B80B670>,
                                              <function mutual_info_classif at 0x00000279A1BABF70>,
                                              <function f_classif at 0x000002799B80B550>],
                         'vec__max_df': [0.8, 0.9, 1.0],
                         'vec__max_features': [10000], 'vec__min_df': [1, 5],
                         'vec__ngram_range': [(1, 1), (1, 2)],
                         'vec__tokenizer': [<function Tokenizer at 0x00000279A6D1C5E

In [150]:
print(grid.best_params_, grid.best_score_)

{'clf__alpha': 0.1, 'clf__fit_prior': False, 'slct__k': 500, 'slct__score_func': <function f_classif at 0x000002799B80B550>, 'vec__max_df': 0.9, 'vec__max_features': 10000, 'vec__min_df': 5, 'vec__ngram_range': (1, 2), 'vec__tokenizer': None} 0.7494608810810692


In [18]:
# Function to add words ending with ת and punctuation counts as specific words to the text 
# so that they are counted as features in the count vectorizer
def add_counts(text):
    new_text = text + ' נגמרבתף ' * count_taf(text)
    new_text = new_text + ' פיסוק ' * count_punct(text)
    return new_text

In [19]:
pipe = Pipeline([
    ('vec', CountVectorizer(max_df=0.9, max_features=10000, min_df=5, ngram_range=(1, 2))),
    ('slct', SelectKBest(k=500, score_func=f_classif)),
    ('clf', MultinomialNB(alpha=0.1, fit_prior=False))
])

cvs = cross_val_score(pipe, X_train.apply(add_counts), y_train, cv=5, scoring='f1_macro')
print(round(cvs.mean(), 2))

0.75


We got a final average f1 macro score of 0.75, the score is lower than before because when cross validating a pipeline the data is split before the preprocessing unlike before where I passed the already preprocessed data to the cross validation.
This score is more meaningful and reliable.

Next i will use the pipe with the parameters found using GridSearchCV to predict the gender in the test dataset.

In [20]:
# Predict the gender in the test dataset
best_pipe = pipe
fitted_pipe = best_pipe.fit(X_train.apply(add_counts), y_train)
pred = fitted_pipe.predict(df_test['story'].apply(add_counts))

# Reverse the encoded gender back to (m, f)
pred = le.inverse_transform(pred)

# Insert ids and predictions into dataframe
df_predicted = pd.DataFrame(data={'test_example_id': df_test['test_example_id'], 'predicted_category':pred})

First 5 Predictions

In [21]:
df_predicted.head(5)

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,m


Last 5 Predictions

In [22]:
df_predicted.tail(5)

,test_example_id,predicted_category
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [23]:
df_predicted.to_csv('classification_results.csv',index=False)